# Importing modules and functions

# 1. Importing modules and functions

In [105]:
import numpy as np
import pandas as pd
from rdkit import Chem, DataStructs
from rdkit.Chem import AllChem, Descriptors
from rdkit.Chem import MACCSkeys
import chembl_structure_pipeline
from molvs import standardize_smiles
from sklearn.model_selection import KFold, GridSearchCV
from sklearn.model_selection import permutation_test_score
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_predict
from sklearn import metrics
from sklearn.metrics import pairwise_distances
import joblib
import pickle
from numpy import savetxt
from padelpy import from_sdf
from IPython.display import HTML
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.ensemble import GradientBoostingRegressor
import warnings
warnings.filterwarnings('ignore')

# 2.Data entry and curation work set

In [106]:
uploaded_file_ws="datasets/HDAC3_work.sdf"
supplier_ws = Chem.ForwardSDMolSupplier(uploaded_file_ws,sanitize=False)
failed_mols_ws = []
all_mols_ws =[]
wrong_structure_ws=[]
wrong_smiles_ws=[]
y_tr = []
y_bad_index=[]

for i, m in enumerate(supplier_ws):
    structure = Chem.Mol(m)
    all_mols_ws.append(structure)
    y_tr.append(m.GetProp("pchembl_value_mean"))
    try:
        Chem.SanitizeMol(structure)
    except:
        failed_mols_ws.append(m)
        wrong_smiles_ws.append(Chem.MolToSmiles(m))
        wrong_structure_ws.append(str(i+1))
        y_bad_index.append(i)
print('Original data: ', len(all_mols_ws), 'molecules')
print('Failed data: ', len(failed_mols_ws), 'molecules')
number_ws =[]
for i in range(len(failed_mols_ws)):
        number_ws.append(str(i+1))
bad_molecules_ws = pd.DataFrame({'No. failed molecule in original set': wrong_structure_ws, 'SMILES of wrong structure: ': wrong_smiles_ws, 'No.': number_ws}, index=None)
bad_molecules_ws = bad_molecules_ws.set_index('No.')
bad_molecules_ws

Original data:  1400 molecules
Failed data:  0 molecules


,No. failed molecule in original set,SMILES of wrong structure:
No.,,


deleting activity values for substances with incorrect structure

In [107]:
y_tr[:] = [x for i,x in enumerate(y_tr) if i not in y_bad_index]

In [108]:
len(y_tr)

1400

# 3.Standardization SDF file for work set

In [109]:
all_mols_ws[:] = [x for i,x in enumerate(all_mols_ws) if i not in y_bad_index] 
records = []
for i in range(len(all_mols_ws)):
    record = Chem.MolToSmiles(all_mols_ws[i])
    records.append(record)

moldf_ws = []
for i,record in enumerate(records):
    standard_record = standardize_smiles(record)
    m = Chem.MolFromSmiles(standard_record)
    moldf_ws.append(m)
    
print('Kept data: ', len(moldf_ws), 'molecules')

Kept data:  1400 molecules


In [110]:
moldf_ws=pd.DataFrame(moldf_ws, columns=['Mol'])
moldf_ws

,Mol
0,<rdkit.Chem.rdchem.Mol object at 0x000002858BD...
1,<rdkit.Chem.rdchem.Mol object at 0x000002859BF...
2,<rdkit.Chem.rdchem.Mol object at 0x000002858BD...
3,<rdkit.Chem.rdchem.Mol object at 0x000002858BD...
4,<rdkit.Chem.rdchem.Mol object at 0x000002858BD...
...,...
1395,<rdkit.Chem.rdchem.Mol object at 0x000002859BF...
1396,<rdkit.Chem.rdchem.Mol object at 0x000002859BF...
1397,<rdkit.Chem.rdchem.Mol object at 0x000002859BF...
1398,<rdkit.Chem.rdchem.Mol object at 0x000002859BF...


# 4.Data entry and curation test set

In [111]:
uploaded_file_ts="datasets/HDAC3_test.sdf"
supplier_ts = Chem.ForwardSDMolSupplier(uploaded_file_ts,sanitize=False)
failed_mols_ts = []
all_mols_ts =[]
wrong_structure_ts=[]
wrong_smiles_ts=[]
y_ts = []
y_bad_index=[]
for i, m in enumerate(supplier_ts):
    structure = Chem.Mol(m)
    all_mols_ts.append(structure)
    y_ts.append(m.GetProp("pchembl_value_mean"))
    try:
        Chem.SanitizeMol(structure)
    except:
        failed_mols_ts.append(m)
        wrong_smiles_ts.append(Chem.MolToSmiles(m))
        wrong_structure_ts.append(str(i+1))
        y_bad_index.append(i)
print('Original data: ', len(all_mols_ts), 'molecules')
print('Failed data: ', len(failed_mols_ts), 'molecules')
number_ts =[]
for i in range(len(failed_mols_ts)):
        number_ts.append(str(i+1))
bad_molecules_ts = pd.DataFrame({'No. failed molecule in original set': wrong_structure_ts, 'SMILES of wrong structure: ': wrong_smiles_ts, 'No.': number_ts}, index=None)
bad_molecules_ts = bad_molecules_ts.set_index('No.')
bad_molecules_ts

Original data:  351 molecules
Failed data:  0 molecules


,No. failed molecule in original set,SMILES of wrong structure:
No.,,


deleting activity values for substances with incorrect structure

In [112]:
y_ts[:] = [x for i,x in enumerate(y_ts) if i not in y_bad_index]

In [113]:
len(y_ts)

351

# 5.Standardization SDF file for test set

In [114]:
all_mols_ts[:] = [x for i,x in enumerate(all_mols_ts) if i not in y_bad_index] 
records = []
for i in range(len(all_mols_ts)):
    record = Chem.MolToSmiles(all_mols_ts[i])
    records.append(record)

moldf_ts = []
for i,record in enumerate(records):
    standard_record = standardize_smiles(record)
    m = Chem.MolFromSmiles(standard_record)
    moldf_ts.append(m)
    
print('Kept data: ', len(moldf_ts), 'molecules')

Kept data:  351 molecules


In [115]:
moldf_ts=pd.DataFrame(moldf_ts, columns=['Mol'])
moldf_ts

,Mol
0,<rdkit.Chem.rdchem.Mol object at 0x000002859BF...
1,<rdkit.Chem.rdchem.Mol object at 0x000002859BF...
2,<rdkit.Chem.rdchem.Mol object at 0x000002859BF...
3,<rdkit.Chem.rdchem.Mol object at 0x00000285945...
4,<rdkit.Chem.rdchem.Mol object at 0x00000285945...
...,...
346,<rdkit.Chem.rdchem.Mol object at 0x000002859BF...
347,<rdkit.Chem.rdchem.Mol object at 0x000002859BF...
348,<rdkit.Chem.rdchem.Mol object at 0x000002859BF...
349,<rdkit.Chem.rdchem.Mol object at 0x000002859BF...


In [116]:
from pathlib import Path

In [124]:
path = Path('feature_name_rfecv_MCCS.txt')
feature_name_rfecv_MACCS = path.read_text().splitlines()

# 6.Calculation MorganFingerprint for work set

In [14]:
def calcfp(mol):
    fp = MACCSkeys.GenMACCSKeys(mol)
    fp = pd.Series(np.asarray(fp))
    fp = fp.add_prefix('MACCS_')
    return fp

# Training set
desc_ws = moldf_ws.Mol.apply(calcfp)
desc_ws

,MACCS_0,MACCS_1,MACCS_2,MACCS_3,MACCS_4,MACCS_5,MACCS_6,MACCS_7,MACCS_8,MACCS_9,...,MACCS_157,MACCS_158,MACCS_159,MACCS_160,MACCS_161,MACCS_162,MACCS_163,MACCS_164,MACCS_165,MACCS_166
0,0,0,0,0,0,0,0,0,0,0,...,0,1,1,1,1,1,1,1,1,0
1,0,0,0,0,0,0,0,0,0,0,...,0,1,1,0,1,1,1,1,1,0
2,0,0,0,0,0,0,0,0,0,0,...,1,1,1,1,1,1,1,1,1,0
3,0,0,0,0,0,0,0,0,0,0,...,0,1,1,1,1,1,1,1,1,0
4,0,0,0,0,0,0,0,0,0,0,...,1,1,1,1,1,1,1,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1395,0,0,0,0,0,0,0,0,0,0,...,1,1,1,1,1,1,1,1,1,0
1396,0,0,0,0,0,0,0,0,0,0,...,1,1,1,1,1,1,1,1,1,0
1397,0,0,0,0,0,0,0,0,0,0,...,1,1,1,1,1,1,1,1,1,0
1398,0,0,0,0,0,0,0,0,1,0,...,0,1,1,1,1,1,1,1,1,0


In [175]:
x_tr=desc_ws[feature_name_rfecv_MACCS]
x_tr

,MACCS_22,MACCS_38,MACCS_42,MACCS_47,MACCS_52,MACCS_53,MACCS_54,MACCS_57,MACCS_62,MACCS_64,...,MACCS_147,MACCS_148,MACCS_149,MACCS_150,MACCS_152,MACCS_153,MACCS_155,MACCS_157,MACCS_160,MACCS_163
0,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,1,0,1,1
1,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,1,0,0,1
2,0,0,0,0,0,1,0,1,1,0,...,1,0,1,0,1,1,1,1,1,1
3,0,0,0,0,0,1,0,0,0,0,...,1,0,0,0,0,0,1,0,1,1
4,0,1,0,1,1,0,0,0,1,1,...,1,1,0,1,0,1,1,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1395,1,1,0,0,0,0,1,0,1,0,...,1,1,1,1,1,1,1,1,1,1
1396,1,1,0,0,0,0,1,0,1,0,...,1,1,1,1,1,1,1,1,1,1
1397,1,1,0,0,0,0,1,0,1,0,...,1,1,1,1,1,1,1,1,1,1
1398,0,1,1,0,0,0,1,1,1,0,...,1,1,0,1,1,1,1,0,1,1


In [172]:
y_tr = np.array(y_tr, dtype=np.float32)
len(y_tr)

1400

# 7.Calculation MorganFingerprint for test set

In [122]:
desc_ts = moldf_ts.Mol.apply(calcfp)
desc_ts

,MACCS_0,MACCS_1,MACCS_2,MACCS_3,MACCS_4,MACCS_5,MACCS_6,MACCS_7,MACCS_8,MACCS_9,...,MACCS_157,MACCS_158,MACCS_159,MACCS_160,MACCS_161,MACCS_162,MACCS_163,MACCS_164,MACCS_165,MACCS_166
0,0,0,0,0,0,0,0,0,0,0,...,0,1,1,1,1,1,1,1,1,0
1,0,0,0,0,0,0,0,0,0,0,...,1,0,1,1,1,1,1,1,1,0
2,0,0,0,0,0,0,0,0,0,0,...,1,1,1,1,1,1,1,1,1,0
3,0,0,0,0,0,0,0,0,0,0,...,1,1,1,1,1,1,1,1,1,0
4,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,1,1,1,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
346,0,0,0,0,0,0,0,0,0,0,...,1,1,1,1,1,1,1,1,1,0
347,0,0,0,0,0,0,0,0,0,0,...,0,1,1,1,1,1,1,1,1,0
348,0,0,0,0,0,0,0,0,0,0,...,0,1,1,1,1,1,1,1,1,0
349,0,0,0,0,0,0,0,0,0,0,...,1,1,1,1,1,1,1,1,1,0


In [176]:
x_ts=desc_ts[feature_name_rfecv_MACCS]
x_ts

,MACCS_22,MACCS_38,MACCS_42,MACCS_47,MACCS_52,MACCS_53,MACCS_54,MACCS_57,MACCS_62,MACCS_64,...,MACCS_147,MACCS_148,MACCS_149,MACCS_150,MACCS_152,MACCS_153,MACCS_155,MACCS_157,MACCS_160,MACCS_163
0,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,1,0,1,1
1,0,0,0,1,0,0,0,0,0,1,...,0,0,0,0,1,1,1,1,1,1
2,0,0,0,0,0,0,1,0,0,0,...,1,0,1,1,0,0,1,1,1,1
3,0,0,0,0,0,0,1,0,1,0,...,0,0,0,1,1,0,1,1,1,1
4,0,1,0,1,0,0,0,0,0,1,...,1,1,0,0,0,1,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
346,1,1,0,0,0,0,1,0,1,0,...,1,1,1,1,1,1,1,1,1,1
347,1,1,0,0,1,0,1,0,1,0,...,1,1,1,1,1,1,1,0,1,1
348,1,1,0,0,0,0,1,0,1,0,...,1,1,1,0,1,1,1,0,1,1
349,1,1,0,0,0,0,1,0,0,0,...,1,1,1,0,1,1,1,1,1,1


In [179]:
y_ts = np.array(y_ts, dtype=np.float32)
y_ts.shape

(351,)

# load the models from disk

In [130]:
best_svm = pickle.load(open('Models/MACCS_FP/HDAC3_SVM_MACCS_FP.pkl', 'rb'))

In [131]:
best_gbr = pickle.load(open('Models/MACCS_FP/HDAC3_GBR_MACCS_final_FS.pkl', 'rb'))

# Prediction for CV

In [132]:
seed = 42
cv=KFold(n_splits=5, random_state=seed, shuffle=True)

In [167]:
x_tr

,MACCS_22,MACCS_38,MACCS_42,MACCS_47,MACCS_52,MACCS_53,MACCS_54,MACCS_57,MACCS_62,MACCS_64,...,MACCS_147,MACCS_148,MACCS_149,MACCS_150,MACCS_152,MACCS_153,MACCS_155,MACCS_157,MACCS_160,MACCS_163
0,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,1,0,1,1
1,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,1,0,0,1
2,0,0,0,0,0,1,0,1,1,0,...,1,0,1,0,1,1,1,1,1,1
3,0,0,0,0,0,1,0,0,0,0,...,1,0,0,0,0,0,1,0,1,1
4,0,1,0,1,1,0,0,0,1,1,...,1,1,0,1,0,1,1,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1395,1,1,0,0,0,0,1,0,1,0,...,1,1,1,1,1,1,1,1,1,1
1396,1,1,0,0,0,0,1,0,1,0,...,1,1,1,1,1,1,1,1,1,1
1397,1,1,0,0,0,0,1,0,1,0,...,1,1,1,1,1,1,1,1,1,1
1398,0,1,1,0,0,0,1,1,1,0,...,1,1,0,1,1,1,1,0,1,1


In [169]:
y_tr.shape

(1400,)

In [133]:
y_pred_CV_svm = cross_val_predict(best_svm, x_tr, y_tr, cv=cv)

In [181]:
y_pred_CV_svm.shape

(1400,)

In [134]:
y_pred_CV_gbr = cross_val_predict(best_gbr, x_tr, y_tr, cv=cv)

In [182]:
y_pred_CV_gbr.shape

(1400,)

In [183]:
y_pred_con=(y_pred_CV_svm+y_pred_CV_gbr)/2

In [184]:
y_pred_con.shape

(1400,)

In [185]:
Q2_CV = round(r2_score(y_tr, y_pred_con), 2)
Q2_CV

0.65

In [186]:
RMSE_CV=round(np.sqrt(mean_squared_error(y_tr, y_pred_con)),2)
RMSE_CV

0.72

# Prediction for test set's molecules

In [187]:
x_ts = np.array(x_ts, dtype=np.float32)
y_ts = np.array(y_ts, dtype=np.float32)

In [188]:
y_pred_svm = best_svm.predict(x_ts)

In [189]:
y_pred_gbr = best_gbr.predict(x_ts)

In [190]:
y_pred_gbr

array([4.76371523, 4.82046158, 6.06388633, 6.1891857 , 4.47094133,
       4.9563409 , 4.94461379, 5.00391796, 5.0221851 , 4.83431854,
       5.88538442, 5.07760087, 5.6165487 , 5.61974321, 5.64596929,
       5.36871546, 4.58848128, 6.00846945, 6.75194491, 4.7522092 ,
       4.9693799 , 7.03672594, 6.21200438, 5.09580898, 5.39539714,
       5.21247438, 5.73897071, 5.01303468, 5.24393999, 5.31002099,
       5.48449245, 5.45613061, 5.61254781, 5.85394981, 5.73354896,
       5.15329142, 6.27245286, 4.95886443, 6.02941375, 5.71083727,
       7.4656184 , 6.58005245, 4.91366103, 6.49308372, 4.86048998,
       6.05239553, 5.6984951 , 5.22151545, 5.92538235, 5.15568388,
       6.46149241, 5.8134084 , 4.09957393, 5.7631971 , 5.14958754,
       6.32600705, 5.76836064, 6.68471684, 6.09804076, 5.1955135 ,
       5.54321911, 6.54693072, 5.24961346, 5.38695918, 5.72587405,
       5.17066536, 5.63266526, 6.42270403, 5.42172807, 5.58124605,
       5.25078086, 5.55598607, 5.79676102, 6.16711111, 5.96062

In [191]:
y_pred_con=(y_pred_svm+y_pred_gbr)/2

In [192]:
Q2_TS = round(r2_score(y_ts, y_pred_con), 2)
Q2_TS

0.69

In [193]:
RMSE_TS=round(np.sqrt(mean_squared_error(y_ts, y_pred_con)), 2)
RMSE_TS

0.69

# Estimating applicability domain. Method - Euclidian distances, K=1

In [194]:
neighbors_k= pairwise_distances(x_tr, n_jobs=-1)
neighbors_k.sort(0)

In [195]:
df_tr=pd.DataFrame(neighbors_k)
df_tr

,0,1,2,3,4,5,6,7,8,9,...,1390,1391,1392,1393,1394,1395,1396,1397,1398,1399
0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,0.000000,0.000000,0.000000,1.000000,2.449490,1.414214,2.449490,2.828427,2.645751,1.732051,...,2.828427,1.732051,1.000000,1.000000,0.000000,1.000000,1.000000,0.000000,3.000000,0.000000
2,1.000000,0.000000,2.645751,2.000000,2.449490,2.645751,2.645751,3.000000,2.828427,2.449490,...,3.316625,2.449490,1.414214,1.000000,1.000000,1.000000,1.000000,0.000000,3.316625,0.000000
3,1.000000,0.000000,2.645751,2.449490,3.000000,3.162278,3.000000,3.000000,3.316625,2.645751,...,3.316625,2.645751,1.414214,1.000000,1.000000,1.000000,1.000000,1.000000,3.605551,1.732051
4,1.000000,1.000000,3.000000,2.645751,3.464102,3.162278,3.000000,3.162278,3.464102,2.645751,...,3.316625,3.162278,1.414214,1.000000,1.000000,1.000000,1.000000,1.000000,3.741657,2.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1395,6.708204,6.708204,6.855655,6.782330,6.633250,7.000000,6.782330,6.403124,7.071068,6.480741,...,6.557439,6.708204,7.141428,7.000000,7.141428,7.280110,7.141428,7.211103,7.000000,7.280110
1396,6.855655,6.708204,6.928203,6.782330,6.633250,7.000000,6.782330,6.480741,7.071068,6.480741,...,6.557439,6.708204,7.141428,7.000000,7.141428,7.280110,7.141428,7.211103,7.000000,7.280110
1397,6.855655,6.708204,6.928203,6.782330,6.708204,7.071068,6.855655,6.480741,7.071068,6.480741,...,6.633250,6.708204,7.141428,7.071068,7.211103,7.348469,7.211103,7.280110,7.071068,7.280110
1398,6.855655,6.782330,7.000000,6.855655,6.708204,7.141428,6.855655,6.480741,7.141428,6.633250,...,6.633250,6.782330,7.141428,7.071068,7.280110,7.416198,7.280110,7.348469,7.071068,7.416198


In [196]:
similarity= neighbors_k

In [197]:
Dmean=np.mean(similarity[1,:])

In [198]:
round(Dmean, 2)

1.31

In [199]:
std=np.std(similarity[1,:])

In [200]:
round(std, 2)

1.07

In [201]:
model_AD_limit=Dmean+std*0.5
print(np.round(model_AD_limit, 2))

1.84


In [202]:
neighbors_k_ts= pairwise_distances(x_tr,Y=x_ts, n_jobs=-1)
neighbors_k_ts.sort(0)

In [203]:
x_ts_AD=pd.DataFrame(neighbors_k_ts)
x_ts_AD

,0,1,2,3,4,5,6,7,8,9,...,341,342,343,344,345,346,347,348,349,350
0,0.000000,2.449490,4.000000,2.000000,1.000000,2.000000,2.449490,1.414214,2.449490,2.449490,...,0.000000,2.645751,1.732051,1.414214,0.000000,0.000000,1.414214,0.000000,2.000000,1.000000
1,0.000000,3.741657,4.123106,2.236068,2.236068,2.236068,2.645751,1.414214,2.645751,2.449490,...,1.000000,2.645751,2.000000,1.732051,0.000000,0.000000,1.414214,1.000000,2.000000,1.414214
2,0.000000,3.872983,4.123106,2.449490,2.449490,3.000000,2.828427,1.732051,2.828427,2.645751,...,2.449490,2.828427,2.000000,1.732051,0.000000,0.000000,1.732051,1.000000,2.236068,2.000000
3,1.000000,3.872983,4.123106,2.645751,3.162278,3.000000,3.464102,2.449490,3.000000,2.645751,...,2.449490,2.828427,2.236068,1.732051,1.732051,1.000000,1.732051,1.000000,2.236068,2.000000
4,1.000000,3.872983,4.123106,2.645751,4.000000,3.162278,3.464102,2.645751,3.162278,2.828427,...,2.645751,3.000000,3.000000,1.732051,2.000000,1.000000,1.732051,1.000000,2.236068,2.236068
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1395,6.708204,6.633250,6.557439,6.480741,7.000000,7.000000,7.141428,7.071068,6.633250,7.141428,...,6.557439,6.480741,6.633250,7.416198,7.280110,7.348469,7.141428,6.855655,7.000000,6.708204
1396,6.782330,6.633250,6.557439,6.480741,7.000000,7.000000,7.141428,7.071068,6.633250,7.141428,...,6.633250,6.480741,6.708204,7.416198,7.280110,7.348469,7.141428,6.928203,7.000000,6.708204
1397,6.782330,6.708204,6.633250,6.633250,7.071068,7.000000,7.141428,7.071068,6.633250,7.141428,...,6.633250,6.557439,6.708204,7.483315,7.280110,7.416198,7.141428,6.928203,7.071068,6.708204
1398,6.855655,6.708204,6.633250,6.633250,7.071068,7.000000,7.141428,7.071068,6.708204,7.211103,...,6.708204,6.557439,6.782330,7.549834,7.416198,7.483315,7.141428,7.071068,7.141428,6.782330


In [204]:
similarity_ts= neighbors_k_ts
cpd_AD=similarity_ts[0,:]
cpd_value = np.round(cpd_AD, 3)
print(cpd_value)

[0.    2.449 4.    2.    1.    2.    2.449 1.414 2.449 2.449 2.236 1.414
 3.464 2.236 2.646 1.    0.    1.414 2.236 2.236 1.    3.    2.    0.
 1.414 1.    1.    0.    1.    1.    3.    1.    2.236 1.414 1.414 1.
 2.    1.414 1.414 2.449 2.646 2.828 2.236 2.828 1.732 1.732 2.236 2.
 1.    1.    2.236 1.414 1.732 0.    1.    3.464 1.    0.    0.    0.
 0.    1.    2.449 3.742 3.162 0.    2.828 2.    0.    2.236 2.236 1.
 1.732 1.414 1.414 0.    1.732 1.    1.732 1.    0.    1.    1.732 0.
 2.236 0.    1.414 1.    0.    3.317 1.    3.464 1.414 2.236 0.    2.449
 1.    0.    0.    1.    3.742 1.414 2.646 0.    1.    0.    2.    0.
 1.414 0.    0.    1.414 1.414 0.    2.449 1.    1.    1.    1.732 1.
 3.    1.414 2.828 1.732 1.    2.646 1.414 1.414 1.    1.    1.414 2.449
 1.    0.    1.414 0.    0.    0.    3.464 1.    3.    1.    1.414 2.449
 0.    1.414 1.    1.    1.414 2.449 2.449 1.414 0.    0.    0.    2.
 1.414 2.236 1.    1.414 2.449 0.    0.    2.236 0.    1.414 1.732 0.
 1.414 1

In [205]:
cpd_AD = np.where(cpd_value <= model_AD_limit, True, False)
print(cpd_AD)

[ True False False False  True False False  True False False False  True
 False False False  True  True  True False False  True False False  True
  True  True  True  True  True  True False  True False  True  True  True
 False  True  True False False False False False  True  True False False
  True  True False  True  True  True  True False  True  True  True  True
  True  True False False False  True False False  True False False  True
  True  True  True  True  True  True  True  True  True  True  True  True
 False  True  True  True  True False  True False  True False  True False
  True  True  True  True False  True False  True  True  True False  True
  True  True  True  True  True  True False  True  True  True  True  True
 False  True False  True  True False  True  True  True  True  True False
  True  True  True  True  True  True False  True False  True  True False
  True  True  True  True  True False False  True  True  True  True False
  True False  True  True False  True  True False  T

In [206]:
print("Coverage = ", sum(cpd_AD) / len(cpd_AD))

Coverage =  0.7236467236467237


In [207]:
print("Indices of substances included in AD = ", np.where(cpd_AD != 0)[0])

Indices of substances included in AD =  [  0   4   7  11  15  16  17  20  23  24  25  26  27  28  29  31  33  34
  35  37  38  44  45  48  49  51  52  53  54  56  57  58  59  60  61  65
  68  71  72  73  74  75  76  77  78  79  80  81  82  83  85  86  87  88
  90  92  94  96  97  98  99 101 103 104 105 107 108 109 110 111 112 113
 115 116 117 118 119 121 123 124 126 127 128 129 130 132 133 134 135 136
 137 139 141 142 144 145 146 147 148 151 152 153 154 156 158 159 161 162
 164 165 166 167 168 169 170 171 172 173 174 177 178 179 180 183 184 185
 186 187 188 189 190 191 192 193 196 197 198 199 201 202 203 204 206 207
 208 209 210 211 213 214 215 216 217 218 220 221 222 223 226 227 228 230
 231 232 233 234 235 237 238 239 240 242 244 245 246 247 249 250 251 252
 253 254 255 256 257 258 259 261 263 264 266 267 268 269 270 271 272 275
 277 280 281 282 283 284 285 286 287 289 291 294 297 298 299 300 301 303
 304 306 307 308 312 313 315 316 317 319 320 321 322 323 324 325 326 327
 328 329 33

In [208]:
out_Ad=list(np.where(cpd_AD == 0)[0])

# Prediction only for molecules included in  AD

In [209]:
y_pred_con_ad=list(y_pred_con)

In [210]:
y_pred_con_ad[:] = [x for i,x in enumerate(y_pred_con_ad) if i not in out_Ad]

In [211]:
len(y_pred_con_ad)

254

In [212]:
y_ts_ad=list(y_ts)

In [213]:
y_ts_ad[:] = [x for i,x in enumerate(y_ts_ad) if i not in out_Ad]

In [214]:
len(y_ts_ad)

254

In [215]:
Q2_TS = round(r2_score(y_ts_ad, y_pred_con_ad), 2)
Q2_TS

0.74

In [216]:
RMSE_TS=round(np.sqrt(mean_squared_error(y_ts_ad, y_pred_con_ad)), 2)
RMSE_TS

0.61